# Membership Inference Attacks (MIAs) 

In this notebook, you will learn how to evaluate and improve the privacy of machine learning models using RMIA from privacy meter tool. 

You will conduct hands-on audits on "locations" tabular dataset and CNN model.

1. Download data and create a config file according data and model.
2. Train the target model for auditing.
3. Compute signals (confidence).
4. Analyze the MIA.


### Environment steup with Pipenv

````
pipenv --python 3.12
````


In [ ]:
# Clone the forked github repo
!git clone https://github.com/<user_name>/ml_privacy_meter.git

In [ ]:
# Change the directory to the cloned repo
import sys
sys.path.append('/content/ml_privacy_meter')

%cd ml_privacy_meter

/Users/tania.carvalho/Desktop/repos/AISec-class/ml_privacy_meter


/Users/tania.carvalho/.local/share/virtualenvs/AISec-class-DqnUEjF1/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
!pipenv install datasets==2.21.0 transformers==4.44.2 torch==2.4.1 torchvision==0.19.1 torchaudio

Delete all CUDA-related lines from requirements.txt if your machine does not support it
```
nvidia-cublas-cu11
nvidia-cudnn-cu11
nvidia-...
triton
```

In [ ]:
!pip install -r requirements.txt

In [18]:
import math
import time
import pandas as pd
import yaml
import tarfile
import zipfile
import os
import urllib.request

import pickle
import numpy as np
import torch
import yaml
from torch.utils.data import Subset

from dataset import TabularDataset
from audit import get_average_audit_results, audit_models, sample_auditing_dataset
from get_signals import get_model_signals
from models.utils import train_models, split_dataset_for_training
from util import (
    check_configs,
    setup_log,
    initialize_seeds,
    create_directories
)

# Enable benchmark mode in cudnn to improve performance when input sizes are consistent
torch.backends.cudnn.benchmark = True

In [ ]:
def write_config_yaml(config_dir: str, filename: str = "config.yaml"):
    """
    Automatically writes a YAML file into a directory.
    Creates the directory if it does not exist.
    """
    os.makedirs(config_dir, exist_ok=True)
    filepath = os.path.join(config_dir, filename)

    config = {
        "run": {
            "random_seed": 12345,
            "log_dir": "demo_locations",
            "time_log": True,
            "num_experiments": 1,
        },

        "audit": {
            "privacy_game": "privacy_loss_model",
            "algorithm": "RMIA",
            "num_ref_models": 1,
            "device": "cpu",
            "report_log": "report_rmia",
            "batch_size": 5000,
            # "data_size": 10000 
        },

        "train": {
            "model_name": "mlp",
            "device": "cpu",
            "batch_size": 256,
            "optimizer": "SGD",
            "learning_rate": 0.1,
            "weight_decay": 0,
            "epochs": 100,
        },
        
        "data": {
            "dataset": "locations",
            "data_dir": "data_locations",
        }
    }

    with open(filepath, "w") as f:
        yaml.dump(config, f, sort_keys=False)

    return filepath

In [17]:
write_config_yaml("configs", "locations_test.yaml")

'configs/locations_test.yaml'

In [ ]:
# Load configs
configs = "configs/locations.yaml"
with open(configs, "rb") as f:
        configs = yaml.load(f, Loader=yaml.Loader)

# Validate configurations
check_configs(configs)

In [6]:
# Validate configurations
check_configs(configs)

# Initialize seeds for reproducibility
initialize_seeds(configs["run"]["random_seed"])

# Create necessary directories
log_dir = configs["run"]["log_dir"]
directories = {
    "log_dir": log_dir,
    "report_dir": f"{log_dir}/report",
    "signal_dir": f"{log_dir}/signals",
    "data_dir": f"{configs["data"]["data_dir"]}",
}
create_directories(directories)

# Set up logger
logger = setup_log(
    directories["report_dir"], "time_analysis", configs["run"]["time_log"]
)

start_time = time.time()

### Locations data

In [7]:
def download_file(url, compression, compressed_file, temp_dir_name):
    urllib.request.urlretrieve(url, compressed_file)

    if compression == 'tar':
        with tarfile.open(compressed_file, 'r:gz') as tar:
            tar.extractall(temp_dir_name, filter='fully_trusted')
        os.remove(compressed_file)

    elif compression == 'zip':
        with zipfile.ZipFile(compressed_file, 'r') as zip_ref:
            zip_ref.extractall(temp_dir_name)
        os.remove(compressed_file)

# Download dataset
url = 'https://github.com/privacytrustlab/datasets/raw/refs/heads/master/dataset_location.tgz'
temp_dir = 'locations_dir'
download_file(url, 'tar', 'dataset_location.tgz', temp_dir)
df = pd.read_csv(os.path.join(temp_dir, 'bangkok'), header=None)
df['Label'] = df.pop(0) # move label to last column


/var/folders/_v/bg3_1x4507x5p5x4xjl3z78r0000gn/T/ipykernel_4437/1177567434.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Label'] = df.pop(0) # move label to last column


In [8]:
# Set up logger
logger = setup_log(
    directories["report_dir"], "time_analysis", configs["run"]["time_log"]
)

path = f"{configs["data"]["data_dir"]}/{configs["data"]["dataset"]}"

df = df.to_numpy()
y = df[:, -1]
X = df[:, :-1].astype(np.float32)
training_size = int(
    len(y) * 0.75
)  # Splitting to create a population dataset
dataset = TabularDataset(X[:training_size], y[:training_size])
population = TabularDataset(X[training_size:], y[training_size:])
with open(f"{path}.pkl", "wb") as file:
    pickle.dump(dataset, file)
logger.info(f"Save data to {path}.pkl")
with open(f"{path}_population.pkl", "wb") as file:
    pickle.dump(population, file)
logger.info(f"Save population data to {path}_population.pkl")

2025-11-28 11:09:40,720 INFO     Save data to data_locations/locations.pkl
2025-11-28 11:09:40,720 INFO     Save data to data_locations/locations.pkl
2025-11-28 11:09:40,731 INFO     Save population data to data_locations/locations_population.pkl
2025-11-28 11:09:40,731 INFO     Save population data to data_locations/locations_population.pkl


Update ouput shape with the tested dataset [number of features, number of classes] -- models/utils.py

```
INPUT_OUTPUT_SHAPE = {
    "cifar10": [3, 10],
    "cifar100": [3, 100],
    "purchase100": [600, 100],
    "texas100": [6169, 100],
    "heart-statlog": [13, 2],
    "locations": [446, 31]
}
```

In [9]:
print("Shape of array:", dataset.data.shape)       # number of rows/columns
print("Number of unique values in target column:", len(np.unique(dataset.targets)))


Shape of array: (3757, 446)
Number of unique values in target column: 30


### Train models --- CNN example

In [10]:
# Define experiment parameters
num_experiments = configs["run"]["num_experiments"]
num_reference_models = configs["audit"]["num_ref_models"]
num_model_pairs = max(math.ceil(num_experiments / 2.0), num_reference_models + 1) # 2 model pairs = 4 models

# train models
baseline_time = time.time()

# Split dataset for training two models per pair
data_splits, memberships = split_dataset_for_training(
    len(dataset), num_model_pairs
)
models_list = train_models(
    log_dir, dataset, data_splits, memberships, configs, logger
)
logger.info(
    "Model loading/training took %0.1f seconds", time.time() - baseline_time
)

2025-11-28 11:09:53,679 INFO     Training 4 models
2025-11-28 11:09:53,679 INFO     Training 4 models
2025-11-28 11:09:53,687 INFO     --------------------------------------------------
2025-11-28 11:09:53,687 INFO     --------------------------------------------------
2025-11-28 11:09:53,689 INFO     Training model 0: Train size 1878, Test size 1879
2025-11-28 11:09:53,689 INFO     Training model 0: Train size 1878, Test size 1879


Using optimizer: SGD | Learning Rate: 0.1 | Weight Decay: 0
Epoch [1/100] | Train Loss: 3.4336 | Train Acc: 0.0367
Test Loss: 3.4282 | Test Acc: 0.0282
Epoch 1 took 17.33 seconds
Epoch [2/100] | Train Loss: 3.4282 | Train Acc: 0.0389
Test Loss: 3.4224 | Test Acc: 0.0314
Epoch 2 took 0.04 seconds
Epoch [3/100] | Train Loss: 3.4237 | Train Acc: 0.0367
Test Loss: 3.4198 | Test Acc: 0.0367
Epoch 3 took 0.02 seconds
Epoch [4/100] | Train Loss: 3.4158 | Train Acc: 0.0426
Test Loss: 3.4120 | Test Acc: 0.0399
Epoch 4 took 0.02 seconds
Epoch [5/100] | Train Loss: 3.4090 | Train Acc: 0.0501
Test Loss: 3.4088 | Test Acc: 0.0442
Epoch 5 took 0.01 seconds
Epoch [6/100] | Train Loss: 3.4060 | Train Acc: 0.0522
Test Loss: 3.4007 | Test Acc: 0.0474
Epoch 6 took 0.02 seconds
Epoch [7/100] | Train Loss: 3.3994 | Train Acc: 0.0564
Test Loss: 3.3999 | Test Acc: 0.0495
Epoch 7 took 0.01 seconds
Epoch [8/100] | Train Loss: 3.3932 | Train Acc: 0.0575
Test Loss: 3.3930 | Test Acc: 0.0479
Epoch 8 took 0.02 sec

2025-11-28 11:10:12,540 INFO     Train accuracy 0.17092651757188498, Train Loss 3.068082332611084
2025-11-28 11:10:12,540 INFO     Train accuracy 0.17092651757188498, Train Loss 3.068082332611084
2025-11-28 11:10:12,541 INFO     Test accuracy 0.14635444385311336, Test Loss 3.1098255813121796
2025-11-28 11:10:12,541 INFO     Test accuracy 0.14635444385311336, Test Loss 3.1098255813121796
2025-11-28 11:10:12,543 INFO     Training model 0 took 18.85512614250183 seconds
2025-11-28 11:10:12,543 INFO     Training model 0 took 18.85512614250183 seconds


Test Loss: 3.1362 | Test Acc: 0.1346
Epoch 85 took 0.01 seconds
Epoch [86/100] | Train Loss: 3.0931 | Train Acc: 0.1571
Test Loss: 3.1325 | Test Acc: 0.1357
Epoch 86 took 0.01 seconds
Epoch [87/100] | Train Loss: 3.0905 | Train Acc: 0.1587
Test Loss: 3.1342 | Test Acc: 0.1357
Epoch 87 took 0.01 seconds
Epoch [88/100] | Train Loss: 3.0904 | Train Acc: 0.1603
Test Loss: 3.1274 | Test Acc: 0.1357
Epoch 88 took 0.01 seconds
Epoch [89/100] | Train Loss: 3.0889 | Train Acc: 0.1608
Test Loss: 3.1312 | Test Acc: 0.1373
Epoch 89 took 0.01 seconds
Epoch [90/100] | Train Loss: 3.0876 | Train Acc: 0.1619
Test Loss: 3.1198 | Test Acc: 0.1373
Epoch 90 took 0.01 seconds
Epoch [91/100] | Train Loss: 3.0845 | Train Acc: 0.1608
Test Loss: 3.1262 | Test Acc: 0.1389
Epoch 91 took 0.01 seconds
Epoch [92/100] | Train Loss: 3.0833 | Train Acc: 0.1645
Test Loss: 3.1247 | Test Acc: 0.1410
Epoch 92 took 0.01 seconds
Epoch [93/100] | Train Loss: 3.0818 | Train Acc: 0.1651
Test Loss: 3.1251 | Test Acc: 0.1410
Epo

2025-11-28 11:10:12,556 INFO     --------------------------------------------------
2025-11-28 11:10:12,556 INFO     --------------------------------------------------
2025-11-28 11:10:12,556 INFO     Training model 1: Train size 1879, Test size 1878
2025-11-28 11:10:12,556 INFO     Training model 1: Train size 1879, Test size 1878


...

2025-11-28 11:12:08,464 INFO     Train accuracy 0.21938232161874335, Train Loss 3.0644866228103638
2025-11-28 11:12:08,464 INFO     Train accuracy 0.21938232161874335, Train Loss 3.0644866228103638
2025-11-28 11:12:08,465 INFO     Test accuracy 0.1974454497072911, Test Loss 3.117405354976654
2025-11-28 11:12:08,465 INFO     Test accuracy 0.1974454497072911, Test Loss 3.117405354976654
2025-11-28 11:12:08,466 INFO     Training model 2 took 58.232645988464355 seconds
2025-11-28 11:12:08,466 INFO     Training model 2 took 58.232645988464355 seconds
2025-11-28 11:12:08,473 INFO     --------------------------------------------------
2025-11-28 11:12:08,473 INFO     --------------------------------------------------
2025-11-28 11:12:08,474 INFO     Training model 3: Train size 1879, Test size 1878
2025-11-28 11:12:08,474 INFO     Training model 3: Train size 1879, Test size 1878


Using optimizer: SGD | Learning Rate: 0.1 | Weight Decay: 0
Epoch [1/100] | Train Loss: 3.4575 | Train Acc: 0.0351
Test Loss: 3.4486 | Test Acc: 0.0447
Epoch 1 took 24.22 seconds
Epoch [2/100] | Train Loss: 3.4509 | Train Acc: 0.0383
Test Loss: 3.4448 | Test Acc: 0.0506
Epoch 2 took 0.02 seconds
Epoch [3/100] | Train Loss: 3.4448 | Train Acc: 0.0431
Test Loss: 3.4358 | Test Acc: 0.0538
Epoch 3 took 0.01 seconds
Epoch [4/100] | Train Loss: 3.4365 | Train Acc: 0.0442
Test Loss: 3.4300 | Test Acc: 0.0564
Epoch 4 took 0.01 seconds
Epoch [5/100] | Train Loss: 3.4292 | Train Acc: 0.0463
Test Loss: 3.4241 | Test Acc: 0.0591
Epoch 5 took 0.01 seconds
Epoch [6/100] | Train Loss: 3.4238 | Train Acc: 0.0506
Test Loss: 3.4203 | Test Acc: 0.0628
Epoch 6 took 0.01 seconds
Epoch [7/100] | Train Loss: 3.4166 | Train Acc: 0.0548
Test Loss: 3.4140 | Test Acc: 0.0682
Epoch 7 took 0.01 seconds
Epoch [8/100] | Train Loss: 3.4089 | Train Acc: 0.0601
Test Loss: 3.4087 | Test Acc: 0.0724
Epoch 8 took 0.01 sec

2025-11-28 11:13:14,126 INFO     Train accuracy 0.2192655667908462, Train Loss 3.063373327255249
2025-11-28 11:13:14,126 INFO     Train accuracy 0.2192655667908462, Train Loss 3.063373327255249
2025-11-28 11:13:14,128 INFO     Test accuracy 0.1900958466453674, Test Loss 3.109704554080963
2025-11-28 11:13:14,128 INFO     Test accuracy 0.1900958466453674, Test Loss 3.109704554080963
2025-11-28 11:13:14,130 INFO     Training model 3 took 65.65716886520386 seconds
2025-11-28 11:13:14,130 INFO     Training model 3 took 65.65716886520386 seconds


Test Loss: 3.1233 | Test Acc: 0.1842
Epoch 92 took 0.01 seconds
Epoch [93/100] | Train Loss: 3.0714 | Train Acc: 0.2118
Test Loss: 3.1214 | Test Acc: 0.1848
Epoch 93 took 0.01 seconds
Epoch [94/100] | Train Loss: 3.0732 | Train Acc: 0.2123
Test Loss: 3.1119 | Test Acc: 0.1858
Epoch 94 took 0.02 seconds
Epoch [95/100] | Train Loss: 3.0656 | Train Acc: 0.2150
Test Loss: 3.1135 | Test Acc: 0.1864
Epoch 95 took 0.02 seconds
Epoch [96/100] | Train Loss: 3.0605 | Train Acc: 0.2161
Test Loss: 3.1160 | Test Acc: 0.1869
Epoch 96 took 0.02 seconds
Epoch [97/100] | Train Loss: 3.0667 | Train Acc: 0.2171
Test Loss: 3.1080 | Test Acc: 0.1874
Epoch 97 took 0.02 seconds
Epoch [98/100] | Train Loss: 3.0622 | Train Acc: 0.2193
Test Loss: 3.1128 | Test Acc: 0.1880
Epoch 98 took 0.01 seconds
Epoch [99/100] | Train Loss: 3.0529 | Train Acc: 0.2193
Test Loss: 3.1121 | Test Acc: 0.1885
Epoch 99 took 0.01 seconds
Epoch [100/100] | Train Loss: 3.0553 | Train Acc: 0.2193
Test Loss: 3.1086 | Test Acc: 0.1901
Ep

2025-11-28 11:13:54,161 INFO     Model loading/training took 240.5 seconds
2025-11-28 11:13:54,161 INFO     Model loading/training took 240.5 seconds


### Auditing dataset

In [19]:
auditing_dataset, auditing_membership = sample_auditing_dataset(
        configs, dataset, logger, memberships
    )

### Compute signals

In [20]:
baseline_time = time.time()
signals = get_model_signals(models_list, auditing_dataset, configs, logger)
population_signals = get_model_signals(
        models_list, population, configs, logger, is_population=True
    )
logger.info("Preparing signals took %0.5f seconds", time.time() - baseline_time)

2025-11-28 11:22:00,379 INFO     Signals loaded from disk successfully.
2025-11-28 11:22:00,379 INFO     Signals loaded from disk successfully.
2025-11-28 11:22:00,385 INFO     Signals loaded from disk successfully.
2025-11-28 11:22:00,385 INFO     Signals loaded from disk successfully.
2025-11-28 11:22:00,386 INFO     Preparing signals took 0.01481 seconds
2025-11-28 11:22:00,386 INFO     Preparing signals took 0.01481 seconds


In [21]:
# Perform the privacy audit
baseline_time = time.time()
target_model_indices = list(range(num_experiments))
mia_score_list, membership_list = audit_models(
        f"{directories['report_dir']}/exp",
        target_model_indices,
        signals,
        population_signals,
        auditing_membership,
        num_reference_models,
        logger,
        configs,
    )

if len(target_model_indices) > 1:
    logger.info(
        "Auditing privacy risk took %0.1f seconds", time.time() - baseline_time
    )

# Get average audit results across all experiments
if len(target_model_indices) > 1:
    get_average_audit_results(
        directories["report_dir"], mia_score_list, membership_list, logger
    )

logger.info("Total runtime: %0.5f seconds", time.time() - start_time)

2025-11-28 11:22:07,522 INFO     Fine-tuning offline_a using paired model 1
2025-11-28 11:22:07,522 INFO     Fine-tuning offline_a using paired model 1
2025-11-28 11:22:07,535 INFO     offline_a=0.00: AUC 0.5654
2025-11-28 11:22:07,535 INFO     offline_a=0.00: AUC 0.5654
2025-11-28 11:22:07,545 INFO     offline_a=0.10: AUC 0.5657
2025-11-28 11:22:07,545 INFO     offline_a=0.10: AUC 0.5657
2025-11-28 11:22:07,552 INFO     offline_a=0.20: AUC 0.5662
2025-11-28 11:22:07,552 INFO     offline_a=0.20: AUC 0.5662
2025-11-28 11:22:07,560 INFO     offline_a=0.30: AUC 0.5666
2025-11-28 11:22:07,560 INFO     offline_a=0.30: AUC 0.5666
2025-11-28 11:22:07,566 INFO     offline_a=0.40: AUC 0.5673
2025-11-28 11:22:07,566 INFO     offline_a=0.40: AUC 0.5673
2025-11-28 11:22:07,572 INFO     offline_a=0.50: AUC 0.5681
2025-11-28 11:22:07,572 INFO     offline_a=0.50: AUC 0.5681
2025-11-28 11:22:07,579 INFO     offline_a=0.60: AUC 0.5694
2025-11-28 11:22:07,579 INFO     offline_a=0.60: AUC 0.5694
2025-11-

<Figure size 640x480 with 0 Axes>

Find the image in ml_privacy_meter/demo_locations/report/exp/ROC_0.png

## Tasks

### 1. Use a different machine learning model (e.g. mlp)
Interpret the new results. Diagnose why the model leaks private information.
Does the model overfit?

### 2. Implement and evaluate defenses. 
Apply a defense strategy (e.g. dropout) to the target model, retrain it and compute the signals again. Analyze the impact of such strategy.